In [1]:
!pip install tensorflow-model-optimization

import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_model_optimization as tfmot
import numpy as np

# Load the caltech101 dataset
dataset_name = "caltech101"
(ds_train, ds_test), ds_info = tfds.load(dataset_name, split=["train[:80%]", "train[80%:]"], as_supervised=True, with_info=True)

# Preprocessing function
IMG_SIZE = 224

def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE)) / 255.0
    return image, label

ds_train = ds_train.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)

# Define a simple CNN model
base_model = tf.keras.applications.MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze base model

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(ds_info.features["label"].num_classes, activation="softmax")
])

# Compile the model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Train the model
model.fit(ds_train, epochs=5, validation_data=ds_test)

# Apply Magnitude-Based Pruning
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

pruning_params = {
    "pruning_schedule": tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.2, final_sparsity=0.8, begin_step=0, end_step=1000),
}

pruned_model = prune_low_magnitude(model, **pruning_params)

# Compile the pruned model
pruned_model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Train the pruned model for fine-tuning
pruned_model.fit(ds_train, epochs=3, validation_data=ds_test, callbacks=[tfmot.sparsity.keras.UpdatePruningStep()])

# Strip pruning for deployment
pruned_model = tfmot.sparsity.keras.strip_pruning(pruned_model)

# Compile the stripped model again before evaluation
pruned_model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]) # Re-compile the stripped model

# Evaluate the pruned model
loss, acc = pruned_model.evaluate(ds_test)
print(f"Pruned model accuracy: {acc:.4f}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 7.4 MB/s eta 0:00:00


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Extraction completed...: 0 file [00:00, ? file/s]

...

Generating train examples...:   0%|          | 0/3060 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/caltech101/incomplete.YIPUM8_3.0.2/caltech101-train.tfrecord*...:   0%|   …

Generating test examples...:   0%|          | 0/6084 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/caltech101/incomplete.YIPUM8_3.0.2/caltech101-test.tfrecord*...:   0%|    …

Dataset caltech101 downloaded and prepared to /root/tensorflow_datasets/caltech101/3.0.2. Subsequent calls will reuse this data.
9406464/9406464 [==============================] - 0s 0us/step
Epoch 1/5
77/77 [==============================] - 23s 183ms/step - loss: 3.1706 - accuracy: 0.3783 - val_loss: 1.2909 - val_accuracy: 0.7386
Epoch 2/5
77/77 [==============================] - 4s 56ms/step - loss: 0.6372 - accuracy: 0.8668 - val_loss: 0.6495 - val_accuracy: 0.8350
Epoch 3/5
77/77 [==============================] - 4s 48ms/step - loss: 0.2286 - accuracy: 0.9608 - val_loss: 0.5216 - val_accuracy: 0.8644
Epoch 4/5
77/77 [==============================] - 5s 59ms/step - loss: 0.1045 - accuracy: 0.9918 - val_loss: 0.4637 - val_accuracy: 0.8824
Epoch 5/5
77/77 [==============================] - 4s 55ms/step - loss: 0.0584 - accuracy: 0.9967 - val_loss: 0.4332 - val_accuracy: 0.8873
Epoch 1/3
77/77 [==============================] - 63s 165ms/step - loss: 0.0662 - accuracy: 0.9914 - val_